In [29]:
import pandas as pd
import gensim

pd.set_option('display.max_colwidth', None)


In [35]:
df = pd.read_csv('aty.csv', index_col=0)
df=df[:100]
df

,Id,QA,Atypical_Annot,Aty,method,obj1,obj2,Atypical_Annot2
0,110991.jpg,I should eat Oreos because I can snack on Oreos during the basketball game,Liquid Deformed Object: The liquid which has been deformed is <b>Milk</b>,milk,Liquid Deformed Object,milk,NaN,Liquid Deformed Object: The liquid which has been deformed is Milk
1,110991.jpg,I should eat Oreos because I can snack on Oreos during the basketball game,Liquid Deformed Object: The liquid which has been deformed is <b>milk</b>,milk,Liquid Deformed Object,milk,NaN,Liquid Deformed Object: The liquid which has been deformed is milk
2,110991.jpg,I should eat Oreos because I can snack on Oreos during the basketball game,Object Replacement: The object which is placed in the context of another object is <b>cookie- milk</b> and the object which is replaced by another object (expected object) <b>person- body- hoop/net</b>,cookie milk person body hoopnet,Object Replacement,cookie- milk,person- body- hoop/net,Object Replacement: The object which is placed in the context of another object is cookie- milk and the object which is replaced by another object (expected object) person- body- hoop/net
3,110991.jpg,I should eat Oreo's because Oreo fun to dunk Oreo.,Liquid Deformed Object: The liquid which has been deformed is <b>Milk</b>,milk,Liquid Deformed Object,milk,NaN,Liquid Deformed Object: The liquid which has been deformed is Milk
4,110991.jpg,I should eat Oreo's because Oreo fun to dunk Oreo.,Liquid Deformed Object: The liquid which has been deformed is <b>milk</b>,milk,Liquid Deformed Object,milk,NaN,Liquid Deformed Object: The liquid which has been deformed is milk
...,...,...,...,...,...,...,...,...
95,13080.jpg,I should buy Absolut vodka because Absolut makes you hungry,Object Replacement: The object which is placed in the context of another object is <b>chocolate box spaces- void in fork- bottle arranged</b> and the object which is replaced by another object (expected object) <b>vodka bottle</b>,vodka bottle,Object Replacement,vodka bottle,NaN,Object Replacement: The object which is placed in the context of another object is chocolate box spaces- void in fork- bottle arranged and the object which is replaced by another object (expected object) vodka bottle
96,13080.jpg,i should get some buy some absolut vodka because some absolut vodka goes with everything,Texture Replacement 2: The object which has a new texture is <b>bottle</b> and the objects that have created the texture are <b>forks- chocolate- food</b>,bottle forks chocolate food,Texture Replacement 2,bottle,forks- chocolate- food,Texture Replacement 2: The object which has a new texture is bottle and the objects that have created the texture are forks- chocolate- food
97,13080.jpg,i should get some buy some absolut vodka because some absolut vodka goes with everything,Object Replacement: The object which is placed in the context of another object is <b>chocolate box spaces- void in fork- bottle arranged</b> and the object which is replaced by another object (expected object) <b>vodka bottle</b>,vodka bottle,Object Replacement,vodka bottle,NaN,Object Replacement: The object which is placed in the context of another object is chocolate box spaces- void in fork- bottle arranged and the object which is replaced by another object (expected object) vodka bottle
98,13080.jpg,I should go out and buy groceries because each image in the ad makes me think of food and beverages.,Texture Replacement 2: The object which has a new texture is <b>bottle</b> and the objects that have created the texture are <b>forks- chocolate- food</b>,bottle forks chocolate food,Texture Replacement 2,bottle,forks- chocolate- food,Texture Replacement 2: The object which has a new texture is bottle and the objects that have created the texture are forks- chocolate- food


In [37]:
def get_keywords(docs):  

    all_stopwords = gensim.parsing.preprocessing.STOPWORDS
    cv=CountVectorizer(max_df=0.85,stop_words=all_stopwords) #,max_features=10000)
    word_count_vector=cv.fit_transform(docs)

    from sklearn.feature_extraction.text import TfidfTransformer

    tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
    tfidf_transformer.fit(word_count_vector)
    feature_names=cv.get_feature_names()

    def sort_coo(coo_matrix):
        tuples = zip(coo_matrix.col, coo_matrix.data)
        return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)

    def extract_topn_from_vector(feature_names, sorted_items, topn=10):
        """get the feature names and tf-idf score of top n items"""

        #use only topn items from vector
        sorted_items = sorted_items[:topn]

        score_vals = []
        feature_vals = []

        for idx, score in sorted_items:
            fname = feature_names[idx]

            #keep track of feature name and its corresponding score
            score_vals.append(round(score, 3))
            feature_vals.append(feature_names[idx])

        #create a tuples of feature,score
        #results = zip(feature_vals,score_vals)
        results= {}
        for idx in range(len(feature_vals)):
            results[feature_vals[idx]]=score_vals[idx]

        return results

    #generate tf-idf for all documents in your list. docs_test has 500 documents
    tf_idf_vector=tfidf_transformer.transform(cv.transform(docs))

    results=[]
    for i in range(tf_idf_vector.shape[0]):

        # get vector for a single document
        curr_vector=tf_idf_vector[i]

        #sort the tf-idf vector by descending order of scores
        sorted_items=sort_coo(curr_vector.tocoo())

        #extract only the top n; n here is 10

        keywords=extract_topn_from_vector(feature_names,sorted_items,10)


        results.append(keywords)
    
    return results



result1 = get_keywords(df['QA'].tolist())
result2 = get_keywords(df['Atypical_Annot2'].tolist())

    
results = pd.DataFrame(zip(df['QA'],result1, df['Atypical_Annot2'], result2),columns=['QA','keywords', 'Atypical_Annot', 'key_words'])
results

,QA,keywords,Atypical_Annot,key_words
0,I should eat Oreos because I can snack on Oreos during the basketball game,"{'oreos': 0.686, 'snack': 0.395, 'game': 0.395, 'basketball': 0.395, 'eat': 0.245}",Liquid Deformed Object: The liquid which has been deformed is Milk,"{'liquid': 0.709, 'deformed': 0.6, 'milk': 0.37}"
1,I should eat Oreos because I can snack on Oreos during the basketball game,"{'oreos': 0.686, 'snack': 0.395, 'game': 0.395, 'basketball': 0.395, 'eat': 0.245}",Liquid Deformed Object: The liquid which has been deformed is milk,"{'liquid': 0.709, 'deformed': 0.6, 'milk': 0.37}"
2,I should eat Oreos because I can snack on Oreos during the basketball game,"{'oreos': 0.686, 'snack': 0.395, 'game': 0.395, 'basketball': 0.395, 'eat': 0.245}",Object Replacement: The object which is placed in the context of another object is cookie- milk and the object which is replaced by another object (expected object) person- body- hoop/net,"{'person': 0.374, 'net': 0.374, 'hoop': 0.374, 'cookie': 0.374, 'body': 0.374, 'milk': 0.257, 'replaced': 0.227, 'placed': 0.227, 'expected': 0.227, 'context': 0.227}"
3,I should eat Oreo's because Oreo fun to dunk Oreo.,"{'oreo': 0.913, 'dunk': 0.264, 'fun': 0.246, 'eat': 0.189}",Liquid Deformed Object: The liquid which has been deformed is Milk,"{'liquid': 0.709, 'deformed': 0.6, 'milk': 0.37}"
4,I should eat Oreo's because Oreo fun to dunk Oreo.,"{'oreo': 0.913, 'dunk': 0.264, 'fun': 0.246, 'eat': 0.189}",Liquid Deformed Object: The liquid which has been deformed is milk,"{'liquid': 0.709, 'deformed': 0.6, 'milk': 0.37}"
...,...,...,...,...
95,I should buy Absolut vodka because Absolut makes you hungry,"{'absolut': 0.739, 'hungry': 0.417, 'vodka': 0.37, 'makes': 0.326, 'buy': 0.193}",Object Replacement: The object which is placed in the context of another object is chocolate box spaces- void in fork- bottle arranged and the object which is replaced by another object (expected object) vodka bottle,"{'bottle': 0.503, 'void': 0.295, 'vodka': 0.295, 'spaces': 0.295, 'fork': 0.295, 'box': 0.295, 'arranged': 0.295, 'chocolate': 0.252, 'replaced': 0.189, 'placed': 0.189}"
96,i should get some buy some absolut vodka because some absolut vodka goes with everything,"{'vodka': 0.647, 'absolut': 0.647, 'goes': 0.365, 'buy': 0.169}",Texture Replacement 2: The object which has a new texture is bottle and the objects that have created the texture are forks- chocolate- food,"{'texture': 0.665, 'forks': 0.358, 'food': 0.358, 'chocolate': 0.306, 'bottle': 0.306, 'created': 0.192, 'new': 0.177, 'replacement': 0.166, 'objects': 0.137}"
97,i should get some buy some absolut vodka because some absolut vodka goes with everything,"{'vodka': 0.647, 'absolut': 0.647, 'goes': 0.365, 'buy': 0.169}",Object Replacement: The object which is placed in the context of another object is chocolate box spaces- void in fork- bottle arranged and the object which is replaced by another object (expected object) vodka bottle,"{'bottle': 0.503, 'void': 0.295, 'vodka': 0.295, 'spaces': 0.295, 'fork': 0.295, 'box': 0.295, 'arranged': 0.295, 'chocolate': 0.252, 'replaced': 0.189, 'placed': 0.189}"
98,I should go out and buy groceries because each image in the ad makes me think of food and beverages.,"{'think': 0.393, 'image': 0.393, 'groceries': 0.393, 'food': 0.393, 'beverages': 0.393, 'ad': 0.319, 'makes': 0.307, 'buy': 0.182}",Texture Replacement 2: The object which has a new texture is bottle and the objects that have created the texture are forks- chocolate- food,"{'texture': 0.665, 'forks': 0.358, 'food': 0.358, 'chocolate': 0.306, 'bottle': 0.306, 'created': 0.192, 'new': 0.177, 'replacement': 0.166, 'objects': 0.137}"
